In [14]:
import math

activities_to_classify = [
        'Get/replace items from refrigerator/cabinets/drawers',
        'Peel a cucumber',
        'Clear cutting board',
        'Slice a cucumber',
        'Peel a potato',
        'Slice a potato',
        'Slice bread',
        'Spread almond butter on a bread slice',
        'Spread jelly on a bread slice',
        'Open/close a jar of almond butter',
        'Pour water from a pitcher into a glass',
        'Clean a plate with a sponge',
        'Clean a plate with a towel',
        'Clean a pan with a sponge',
        'Clean a pan with a towel',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
        'Stack on table: 3 each large/small plates, bowls',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
    ]

activities_to_verbs = {
        'Get/replace items from refrigerator/cabinets/drawers': 'Get',
        'Peel a cucumber': 'Peel',
        'Clear cutting board': 'Clear',
        'Slice a cucumber': 'Slice',
        'Peel a potato': 'Peel',
        'Slice a potato': 'Slice',
        'Slice bread': 'Slice',
        'Spread almond butter on a bread slice': 'Spread',
        'Spread jelly on a bread slice': 'Spread',
        'Open/close a jar of almond butter': 'Open/Close',
        'Pour water from a pitcher into a glass': 'Pour',
        'Clean a plate with a sponge': 'Clean',
        'Clean a plate with a towel': 'Clean',
        'Clean a pan with a sponge': 'Clean',
        'Clean a pan with a towel': 'Clean',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Get',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Set',
        'Stack on table: 3 each large/small plates, bowls': 'Stack',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load'
}

verbs = {
    'Get': 0,
    'Peel': 1,
    'Clear': 2,
    'Slice': 3,
    'Spread': 4,
    'Open/Close': 5,
    'Pour': 6,
    'Clean': 7,
    'Set': 8,
    'Stack': 9,
    'Load': 10,
}



def fill_missing_intervals(df):
    df = df.sort_values(by='start_frame')

    dfs = []

    for i in range(len(df) - 1):
        current_row = df.loc[i]
        next_row = df.loc[i + 1]

        missing_interval_start = current_row['stop_frame'] + 1
        missing_interval_end = next_row['start_frame'] - 1

        if missing_interval_start <= missing_interval_end:
            new_row = pd.DataFrame({
                'uid': 90,
                'participant_id': 'S04',
                'video_id': 'S04_01',
                'narration': 'None',
                'verb': 'None',
                'verb_class': 0,
                'start_frame': [missing_interval_start],
                'stop_frame': [missing_interval_end],
                'start_timestamp': [missing_interval_start / 22],
                'stop_timestamp': [missing_interval_end / 22],
            })
            dfs.append(new_row)

    if dfs:
        filled_df = pd.concat([df] + dfs, ignore_index=True)
    else:
        filled_df = df.copy()

    return filled_df

def split_rows(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = 110  

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/22
            new_row['stop_timestamp'] = new_stop/22
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def extend_video_id(df):
    extended_data = []
    num_extensions = 4

    for _ in range(num_extensions):
        extended_data.append(df.copy())

    for i in range(num_extensions):
        extended_data[i]['video_id'] = f'S04_0{i + 2}' 

    result = pd.concat([df] + extended_data, ignore_index=True)
    return result


def create_annotations_file(timestamp_file, annotations_file_test, annotations_file_train):
    import pandas as pd

    video_fps = 22

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_timestamp'] = timestamps['start'] - start_timestamp
    timestamps['stop_timestamp'] = timestamps['stop'] - start_timestamp
    timestamps['start_frame'] = (timestamps['start_timestamp'] * video_fps).astype(int)
    timestamps['stop_frame'] = (timestamps['stop_timestamp'] * video_fps).astype(int)
    
    data_frame_train = pd.read_pickle(annotations_file_train)
    data_frame_train = data_frame_train[data_frame_train['file'] == 'S04_1.pkl']
    data_frame_train = data_frame_train.drop(['labels'], axis=1)
    data_frame_train['type'] = 'train'

    data_frame_test = pd.read_pickle(annotations_file_test)
    data_frame_test = data_frame_test[data_frame_test['file'] == 'S04_1.pkl']
    data_frame_test = data_frame_test.drop(['labels'], axis=1)
    data_frame_test['type'] = 'test'
    
    
    join_data_frame = pd.concat([data_frame_train, data_frame_test], ignore_index=True)
    join_data_frame.to_csv("JOINED.csv", index=False)

    result = pd.merge(timestamps, join_data_frame, on='index', how='inner')

    result['uid'] = result['index']
    result['participant_id'] = 'S04'
    result['video_id'] = 'S04_01'
    result['verb'] = result['description_x'].map(activities_to_verbs)
    result['narration'] = result['description_x']
    result['verb_class'] = result['verb'].map(verbs).map(int)

    result = result[
        ['uid', 'participant_id', 'video_id', 'narration', 'start_timestamp', 'stop_timestamp', 'start_frame',
         'stop_frame', 'verb', 'verb_class', 'type']]
    
    
    #result = fill_missing_intervals(result)
    
    result = result.sort_values(by='start_frame')
    result['uid'] = range(len(result))
    
    train_proportion = 0.7

    none_rows = result[result['type'].isna()]

    num_train = int(len(none_rows) * train_proportion)

    train_indices = none_rows.sample(n=num_train, random_state=42).index
    result.loc[train_indices, 'type'] = 'train'

    test_indices = none_rows.drop(train_indices).index
    result.loc[test_indices, 'type'] = 'test'
    
    
    result = extend_video_id(result)
    result = split_rows(result)
    result['uid'] = range(len(result))
    
    
    test = result[result['type'] == 'test']
    test.to_csv("S04_test.csv", index=False)
    test.to_pickle("S04_test.pkl")
    train = result[result['type'] == 'train']
    train.to_csv("S04_train.csv", index=False)
    train.to_pickle("S04_train.pkl")
    
    
    result.to_pickle(f"JOINED.pkl")
    result.to_csv(f"JOINED.csv", index=False)


In [15]:
create_annotations_file('S04_1.pkl', 'action-net/ActionNet_test.pkl', 'action-net/ActionNet_train.pkl')

In [13]:
import pandas as pd
df = pd.read_pickle('S04_train_other.pkl')

df

,uid,participant_id,video_id,narration,start_timestamp,stop_timestamp,start_frame,stop_frame,verb,verb_class
9,10,S04,S04_1,Get/replace items from refrigerator/cabinets/d...,1413.433146,1483.612856,42402,44508,Get,0
5,6,S04,S04_1,Slice a cucumber,1176.920691,1225.454613,35307,36763,Slice,3
16,17,S04,S04_1,Slice a potato,1787.514367,1811.889299,53625,54356,Slice,3
43,44,S04,S04_1,Clean a plate with a sponge,2624.026289,2628.679281,78720,78860,Clean,7
36,37,S04,S04_1,Pour water from a pitcher into a glass,2423.629084,2428.528071,72708,72855,Pour,6
26,27,S04,S04_1,Spread almond butter on a bread slice,2162.605734,2177.567708,64878,65327,Spread,4
48,49,S04,S04_1,Clean a pan with a sponge,2673.609262,2677.756256,80208,80332,Clean,7
32,33,S04,S04_1,Open/close a jar of almond butter,2303.492563,2311.946915,69104,69358,Open/Close,5
28,29,S04,S04_1,Spread jelly on a bread slice,2216.746695,2231.433670,66502,66943,Spread,4
23,24,S04,S04_1,Get/replace items from refrigerator/cabinets/d...,2043.755944,2109.551838,61312,63286,Get,0


In [4]:
import pandas as pd
df = pd.read_pickle('action_net_annotations/S04_train.pkl')

df

,uid,participant_id,video_id,narration,start_timestamp,stop_timestamp,start_frame,stop_frame,verb,verb_class,type
0,0,S04,S04_01,Get/replace items from refrigerator/cabinets/d...,860.227273,865.181818,18925,19034,Get/replace items from refrigerator/cabinets/d...,0,train
1,1,S04,S04_01,Get/replace items from refrigerator/cabinets/d...,865.227273,870.181818,19035,19144,Get/replace items from refrigerator/cabinets/d...,0,train
2,2,S04,S04_01,Get/replace items from refrigerator/cabinets/d...,870.227273,875.181818,19145,19254,Get/replace items from refrigerator/cabinets/d...,0,train
3,3,S04,S04_01,Get/replace items from refrigerator/cabinets/d...,875.227273,880.181818,19255,19364,Get/replace items from refrigerator/cabinets/d...,0,train
4,4,S04,S04_01,Get/replace items from refrigerator/cabinets/d...,880.227273,885.181818,19365,19474,Get/replace items from refrigerator/cabinets/d...,0,train
...,...,...,...,...,...,...,...,...,...,...,...
2090,2090,S04,S04_05,"Unload dishwasher: 3 each large/small plates, ...",3361.772727,3366.727273,73959,74068,"Unload dishwasher: 3 each large/small plates, ...",19,train
2091,2091,S04,S04_05,"Unload dishwasher: 3 each large/small plates, ...",3366.772727,3371.727273,74069,74178,"Unload dishwasher: 3 each large/small plates, ...",19,train
2092,2092,S04,S04_05,"Unload dishwasher: 3 each large/small plates, ...",3371.772727,3376.727273,74179,74288,"Unload dishwasher: 3 each large/small plates, ...",19,train
2093,2093,S04,S04_05,"Unload dishwasher: 3 each large/small plates, ...",3376.772727,3381.727273,74289,74398,"Unload dishwasher: 3 each large/small plates, ...",19,train
